In [1]:
import numpy as np
import cv2
import math as m
from matplotlib import pyplot as plt

In [2]:
originalImg = cv2.imread('castle.jpg')
gray = originalImg[:,:,0]

In [3]:
def zero_pad (image, pad):
    paddedImage = np.pad(image,((pad,pad),(pad,pad)), 'constant', constant_values = (0,0))
    return paddedImage

In [4]:
def conv_single_step(imageSlice, window):
    s = np.multiply(imageSlice,window)
    Z = np.sum(s)
    return Z

In [5]:
def convolution(image, window, pad):
    window = np.flip(window, axis = 1)
    window = np.flip(window, axis = 0)
    paddedImage = zero_pad(image, pad)                    # pad the image
    (paddedImageH, paddedImageW) = np.shape(paddedImage)  # retrieve dimensions of padded image
    (outputH, outputW) = np.shape(image)                  # retrieve dimensions of image
    (windowH, windowW) = np.shape(window)                 #retrieve shape of window
    Z = np.zeros((outputH, outputW))                        # inialize the output volume Z with zeros
    for h in range(outputH):                              # loop over vertical axis of the output volume
            for w in range(outputW):                      # loop over horizontal axis of the output volume
                vert_start = h                            # find the corners of the current slice
                vert_end = h+windowH
                horiz_start = w
                horiz_end = w+windowW
                imageSlice = paddedImage[vert_start:vert_end, horiz_start:horiz_end] # get the value for the slice
                Z[h,w] = conv_single_step(imageSlice, window)                        # do convolution over the current slice
    Z=Z.astype(np.uint8)
    return Z
                

In [6]:
def gaussianfilter(windowSize, k, sigma):                 #window = window size 
    window = np.random.rand(windowSize, windowSize)
    (windowH, windowW) = np.shape(window)                 #retrieve shape of window
    kernel = np.zeros((windowH, windowW))
    for h in range(windowH):                             
            for w in range(windowW):                  
                kernel[h,w] = k*(m.exp(-((h-(windowH/2-0.5))**2+(w-(windowW/2-0.5))**2)/(2*sigma**2)))
    windowSum = np.sum(kernel)
    kernel = kernel/windowSum
    return kernel

In [7]:
gWindow = gaussianfilter(3, 1, 1)
#print (yg)
gaussianImage = convolution(gray, gWindow, 1)

In [ ]:
#np.set_printoptions(threshold=np.nan)
#print(gWindow)
#print(gray)
#print(gaussianImage)

In [8]:
numpy_horizontal = np.hstack((gray, gaussianImage))
cv2.imshow('original vs Gaussian Image',numpy_horizontal)
#cv2.imshow('image',gray)
#cv2.imshow('image',gaussianImage)
#cv2.imshow('l' , np.array(gaussianImage, dtype = np.uint8 ) )
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
def laplacian(c):
    l=np.zeros((3,3))
    l[1,1] = c
    l[0,1] = -c/4
    l[1,0] = -c/4
    l[1,2] = -c/4
    l[2,1] = -c/4
    return l

In [10]:
def laplacianfilter(image, c, pad):
    l = laplacian(c)
    paddedImage = zero_pad(image, pad)                    # pad the image
    (paddedImageH, paddedImageW) = np.shape(paddedImage)  # retrieve dimensions of padded image
    (outputH, outputW) = np.shape(image)                  # retrieve dimensions of image
    (windowH, windowW) = np.shape(l)                      # retrieve shape of window
    Z = np.zeros((outputH, outputW))                      # inialize the output volume Z with zeros
    for h in range(outputH):                              # loop over vertical axis of the output volume
            for w in range(outputW):                      # loop over horizontal axis of the output volume
                vert_start = h                            # find the corners of the current slice
                vert_end = h+windowH
                horiz_start = w
                horiz_end = w+windowW
                imageSlice = paddedImage[vert_start:vert_end, horiz_start:horiz_end] # get the value for the slice
                g = conv_single_step(imageSlice, l)                                  # do convolution to get gradient
                Z[h,w] = image[h,w]+c*g
    Z=Z.astype(np.uint8)
    return Z

In [11]:
laplacianImage = laplacianfilter(gaussianImage, 0.1, 1)    #Laplacian of Gaussian

In [12]:
numpy_horizontal2 = np.hstack((gaussianImage, laplacianImage))
cv2.imshow('Gaussian Image vs Laplacian of Gaussian Image',numpy_horizontal2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [13]:
laplacianImage2 = laplacianfilter(gray, 0.1, 1)                 #Laplacian of original image

In [14]:
numpy_horizontal3 = np.hstack((gray, laplacianImage2))
cv2.imshow('original image vs Laplacian of original',numpy_horizontal3)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
numpy_horizontal4 = np.hstack((laplacianImage, laplacianImage2))
cv2.imshow('Laplacian of Gaussian Image vs Laplacian of original',numpy_horizontal4)
cv2.waitKey(0)
cv2.destroyAllWindows()